In [ ]:
################################
########### IMPORTS ############
################################

In [1]:
from pycbc import distributions
from pycbc.waveform import get_td_waveform, td_approximants
from pycbc.detector import Detector
import matplotlib.pyplot as plt
import numpy as np
import gwpy
import pylab
from tqdm.notebook import tqdm
from gwpy.timeseries import TimeSeries
import pandas as pd
import os
import csv
import pycbc.noise
import pycbc.psd
from pycbc.filter import matched_filter

In [2]:
print("Generated Binary Mass Distributions for BBH")
bbh_two_mass_distributions = distributions.Uniform(mass1=(10, 50),
                                               mass2=(10, 50))

bbh_two_mass_samples = bbh_two_mass_distributions.rvs(size=1000)

print("Generated Binary Mass Distributions for BNS")
bns_two_mass_distributions = distributions.Uniform(mass1=(1, 2),
                                               mass2=(1, 2))

bns_two_mass_samples = bns_two_mass_distributions.rvs(size=1000)

Generated Binary Mass Distributions for BBH
Generated Binary Mass Distributions for BNS


In [3]:
for i in tqdm(range(1000)):
    
    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)

    noise *= 1e-17
    noise.write("/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/noise_templates/noise_4k_"+str(i)+".txt")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
start_times_bbh = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5]
start_times_bns = [0, 0.5, 1, 1.5, 2, 2.5, 3]

for i in tqdm(range(len(bbh_two_mass_samples))):

    hp1, hc1 = get_td_waveform(approximant="SEOBNRv2",                                
                         mass1=bbh_two_mass_samples[i][0],
                         mass2=bbh_two_mass_samples[i][1],
                         delta_t=1.0/4096,
                         f_lower=40)


    hp2, hc2 = get_td_waveform(approximant="IMRPhenomPv2_NRTidal", 
                         mass1=bns_two_mass_samples[i][0],
                         mass2=bns_two_mass_samples[i][1],
                         delta_t=1.0/4096,
                         f_lower=40)


    bbh_signal = TimeSeries.from_pycbc(hp1)
    st1 = np.random.randint(0, 8)
    bbh_signal.t0 = start_times_bbh[st1]
    bbh_signal = (bbh_signal/(max(bbh_signal.max(), np.abs(bbh_signal.min()))))*0.2


    # Extract the last 1 sec from the BNS signal
    t = hp2.get_end_time()
    hp2 = hp2.time_slice(t-1, t)

    bns_signal = TimeSeries.from_pycbc(hp2)
    st2 = np.random.randint(0, 7)
    bns_signal.t0 = start_times_bns[st2]
    bns_signal = bns_signal.taper()
    bns_signal = (bns_signal/(max(bns_signal.max(), np.abs(bns_signal.min()))))*0.2

    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)


    data1 = noise.inject(bbh_signal)
    data2 = noise.inject(bns_signal)

    data1 *= 1e-17
    data2 *= 1e-17
    
    data1.write("/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/merged_bbh_noise_signal/merged_noise_signal_"+str(i)+".txt")
    data2.write("/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/merged_bns_noise_signal/bns_merged_noise_signal_"+str(i)+".txt")

In [4]:
# Alternate
start_times_bbh = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5]
start_times_bns = [0, 0.5, 1, 1.5, 2, 2.5, 3]

for i in tqdm(range(1000)):

    bbh_signal = TimeSeries.read("/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/bbh_signal/bbh_4k_"+str(i)+".txt")
    st1 = np.random.randint(0, 8)
    bbh_signal.t0 = start_times_bbh[st1]
    bbh_signal = (bbh_signal/(max(bbh_signal.max(), np.abs(bbh_signal.min()))))*0.2


    bns_signal = TimeSeries.read("/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/bns_signal/bns_signal_"+str(i)+".txt")
    st2 = np.random.randint(0, 7)
    bns_signal.t0 = start_times_bns[st2]
    bns_signal = bns_signal.taper()
    bns_signal = (bns_signal/(max(bns_signal.max(), np.abs(bns_signal.min()))))*0.2

    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)


    data1 = noise.inject(bbh_signal)
    data2 = noise.inject(bns_signal)

    data1 *= 1e-17
    data2 *= 1e-17
    
    data1.write("/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/merged_bbh_noise_signal/merged_noise_signal_"+str(i)+".txt")
    data2.write("/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/merged_bns_noise_signal/bns_merged_noise_signal_"+str(i)+".txt")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
# Merging BBH Noise + Signal Templates into single csv file

path = "/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/merged_bbh_noise_signal/"
files= os.listdir(path)
f = open('/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/val_Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
# Merging Noise Templates into single csv file

path_1 = "/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/noise_templates/"
files_1= os.listdir(path_1)
f1 = open('/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/val_Final_Merged_Noise_Reduced_No_Abs.csv', 'w')
cw_1 = csv.writer(f1)

for i in tqdm(files_1):
    #print(files)
    df = pd.read_csv(path_1+i,sep = ' ', header=None)
    c = df[:][1]
    cw_1.writerow(c)
f1.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
# Merging BNS Noise + Signal Templates into single csv file

path = "/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/merged_bns_noise_signal/"
files= os.listdir(path)
f = open('/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/raw_val_data_files/val_Final_BNS_Merged_Noise_Signal_Reduced_No_ABS.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/1000 [00:00<?, ?it/s]